# 2. Data Preparation

a. Handle missing or erroneous data. <br>
b. Encode categorical variables appropriately. <br>
c. Normalize or standardize features as required. <br>
d. Split the data into training and testing sets to evaluate model performance. <br>

#### Importing the relevant modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

#### 2a. Handle missing or erroneous data. <br>

In [ ]:
csv_path = Path("dataset.csv")
df = pd.read_csv(csv_path)

# Total number of rows in the dataset with all the fields filled - No missing values
df_full = df.dropna()

print("Original shape:", df.shape)
print("Full rows shape:", df_full.shape)

Original shape: (20501, 15)
Full rows shape: (9531, 15)


In [ ]:
# Total number of columns of the dataset having No row as missing
df.dropna(axis=1)

,Time_step
0,15/03/2022 10:24
1,15/03/2022 10:24
2,15/03/2022 10:24
3,15/03/2022 10:24
4,15/03/2022 10:24
...,...
20496,17/06/2024 11:54
20497,17/06/2024 12:04
20498,17/06/2024 12:14
20499,
